In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv008569/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [4]:
base_order=spark.sparkContext.textFile("/public/trendytech/retail_db/orders")
base_order_2=base_order.map(lambda x: (int (x.split(",")[0]),int(x.split(",")[2])))
base_order_detail_rdd= spark.sparkContext.textFile("/public/trendytech/retail_db/order_items")
base_order_selected_rdd=base_order_detail_rdd.map(lambda x: (int(x.split(",")[1]),float(x.split(",")[4])))
base_rdd_join=base_order_selected_rdd.join(base_order_2)
final_rdd=base_rdd_join.map(lambda x:(x[1][1],x[1][0]))
reduce_rdd=final_rdd.reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1],ascending=False)
#reduce_rdd.take(10) ##Q1 answer
base_order_Q2=base_order_detail_rdd.map(lambda x: (int(x.split(",")[2]),int(x.split(",")[3])))
reduced_base_order_Q2=base_order_Q2.reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1],ascending=False)
#reduced_base_order_Q2.take(10)  ##Q2 answer
base_customer_detail_rdd= spark.sparkContext.textFile("/public/trendytech/retail_db/customers")
base_customer_detail_Q3=base_customer_detail_rdd.map(lambda x:( x.split(",")[6]))
base_customer_detail_Q3_Ans=base_customer_detail_Q3.filter(lambda x: x=='Caguas')
base_customer_detail_Q3_Ans.count() ##Q3 answer
base_customer_detail_Q4=base_customer_detail_rdd.map(lambda x:( x.split(",")[7],1))
base_customer_detail_Q4_ans=base_customer_detail_Q4.reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1],ascending=False)
#base_customer_detail_Q4_ans.take(3) ##Q4 answer
reduce_rdd_Q5=reduce_rdd.filter(lambda x: x[1]>1000)
#reduce_rdd_Q5.count()  ##Q5 answer
base_order_Q5_rdd=base_order.map(lambda x:  ((x.split(",")[2]),(x.split(",")[3])))
base_order_filter_Q5=base_order_Q5_rdd.filter(lambda x: x[1]=='CLOSED')
base_customer_Q5=base_customer_detail_rdd.map(lambda x: ((x.split(",")[0]),( x.split(",")[7])))
base_order_join_Q5=base_customer_Q5.join(base_order_filter_Q5)
final_rdd_Q5=base_order_join_Q5.map(lambda x: (x[1][0],1))
final_rdd_state= final_rdd_Q5.reduceByKey( lambda x,y: x+y).sortBy(lambda x:x[1], ascending=False)
base_order_Q56rdd=base_order.map(lambda x:  ((x.split(",")[2],1)))
base_order_Q6=base_order_Q56rdd.reduceByKey(lambda x,y : x+y)
base_order_Q6.count() #Q6 answer

base_order_Q7=base_order_detail_rdd.map(lambda x: (int(x.split(",")[0]),int(x.split(",")[1]))) #Order_Id, customer_id
base_customer_detail_Q7=base_customer_detail_rdd.map(lambda x:((int( x.split(",")[0])),( x.split(",")[7]))) # customer_id,customer_state
base_order_customer_join_Q7=base_order_Q7.join(base_customer_detail_Q7).map(lambda x: (x[0],x[1][1])) #Order_id, customer_state
base_order_selected_Q7=base_order_detail_rdd.map(lambda x: (int(x.split(",")[1]),float(x.split(",")[4]))) #Order id,order_item_subtotal
final_rdd_Q7=base_order_customer_join_Q7.join(base_order_selected_Q7).map (lambda x: (x[1][0],x[1][1])).reduceByKey(lambda x,y : x+y).sortBy(lambda x:x[1], ascending=False) #order id ,customer_state, order_item_subtotal
final_rdd_Q7.collect() #Q7 answer0


12405

In [19]:
ake(5)
#Q7_join=base_customer_detail_Q7.join(base_order_selected_rdd).map(lambda x : (x[1][0],x[1][1]))
#Q7_join_state=Q7_join.reduceByKey(lambda x,y: x+y).sortBy(lambda x:x[1], ascending=False)
#Q7_join_state.take(10)
#base_order_selected_rdd.take(5)
#base_order_selected_Q7=base_order_detail_rdd.map(lambda x: (int(x.split(",")[1]),float(x.split(",")[4])))

[('PR', 2374562.529999976),
 ('CA', 966514.4300000025),
 ('NY', 398774.0800000004),
 ('TX', 297226.76000000007),
 ('IL', 251831.33000000016),
 ('FL', 180943.52000000008),
 ('PA', 138451.59999999998),
 ('OH', 130595.46999999999),
 ('MI', 124369.04999999993),
 ('NJ', 111204.99999999994),
 ('AZ', 106321.23999999996),
 ('GA', 85722.04999999994),
 ('NC', 76741.75999999994),
 ('VA', 72012.34999999996),
 ('MD', 70963.35999999996),
 ('OR', 63527.52999999996),
 ('MA', 59140.47999999997),
 ('CO', 55927.83999999998),
 ('NV', 53240.869999999966),
 ('TN', 50566.10999999997),
 ('MO', 45753.709999999985),
 ('HI', 44647.01999999998),
 ('CT', 38686.229999999996),
 ('NM', 35113.219999999994),
 ('WA', 34325.36999999999),
 ('UT', 32113.09999999999),
 ('WI', 31983.46),
 ('LA', 31708.119999999988),
 ('SC', 26875.75),
 ('DC', 24573.909999999996),
 ('KY', 18573.56),
 ('IN', 17781.659999999996),
 ('MN', 17132.98),
 ('KS', 12533.579999999998),
 ('DE', 11056.720000000001),
 ('OK', 8254.91),
 ('ND', 7813.09000000